통계 기반 vs 추론 기반
- 학습 방법의 차이 
- 새로운 단어가 추가되는 경우 추론 기반이 단어의 분산 표현을 효율적으로 갱신 할 수 있음 
- 통계 기반은 단어의 유사성이 추론 기반은 단어의 유사성과 단어 사이의 패턴까지 인코딩 (But 성능 차이 X)
- skip-gram 과 네거티브 샘플리을 이용한 모델은 모두 말뭉치 전체의 동시발생 행렬에 특수한 행렬 분해를 적용한 것 
- GloVe는 말뭉치 전체의 통계 정보를 손실 함수에 도입해 미니배치를 학습하는 두 기반의 융합한 기법임

#### word2vec 문제점 : 말뭉치에 포함된 어휘 수가 많아지면 계산량도 커짐 

##### 속도 개선 
1) Embedding 계층
2) 네커티브 샘플링

In [5]:
from common.layers import Embedding, SigmoidWithLoss
import collections

In [3]:
import numpy as np
#Embedding 계층 구현
class Embedding:
    def __init__(self,W):
        self.params= [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self,idx):
        W, =self.params
        self.idx = idx 
        out = W[idx]
        return out 
    
    def backward(self,dout):
        dW,=self.grads 
        dW[...]=0
        
        np.add.at(dW,self.idx,dout)

#### 네거티브 샘플링 (시그모이드 함수+교차 엔트로피 오차)
- 긍정적 예를 타킷으로 한 경우의 손실의 구함 + 동시에 적은 수의 부정적 예를 샘플링 해서 사용

다중분류 -> 이중분류

In [4]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh


In [13]:
#말뭉치에서 단어의 확률분포를 만들고 다시 0.75를 제곱한 다음 np.random.choice()를 사용해 부정적 예 샘플링함 
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [17]:
GPU=False
corpus=np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size =2 

sampler=UnigramSampler(corpus,power,sample_size)
target =np.array([1,3,0])
negative_sample = sampler.get_negative_sample(target)
negative_sample

array([[2, 3],
       [4, 0],
       [2, 3]])

In [18]:
#negative sampling 구현
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh
